<a href="https://colab.research.google.com/github/minjikim13/career-event-attendance-cohort-analysis/blob/main/Career_Event_SQL_Cohort_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Cohort Analysis - Career Event Attendance

This notebook analyses attendance behaviour using SQL-based cohort analysis.

Cohorts are defined by:
- Registration timing
- Country
- University
- Study area

The analysis aims to identify drivers of no-shows and opportunities to improve attendance rates of next events.


In [1]:
from google.colab import files

uploaded = files.upload()

Saving Final_cleaned_data.xlsx to Final_cleaned_data.xlsx


In [3]:
import pandas as pd
import numpy as np
import sqlite3

df = pd.read_excel('Final_cleaned_data.xlsx')
df.head()

,Order,Order Date,First Name,Last Name,Country,University,Area of Study,Attended,Question
0,13706070183,2025-11-12 15:08:34,Arnob,Roy,Bangladesh,Kaplan Business School,Accounting,0,NaN
1,13654297383,2025-11-05 18:12:10,Sumitra,Gurung,Nepal,university of newcastle,Accounting,0,NaN
2,13678792843,2025-11-08 14:32:05,Cassie,Liu,China,UNSW,Accounting,0,NaN
3,13809389593,2025-11-27 14:47:04,Suryansh,Singh,India,USYD,Accounting,0,How to start you career as an international st...
4,13756192373,2025-11-18 12:38:23,Yonghui,Meng,China,UTS,Accounting,0,"The arrangements about the activity, timeline"


In [4]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
event_date = df['Order Date'].max()
df['Days_Before'] = (event_date - df['Order Date']).dt.days

In [5]:
def timing_group(days):
    if days <= 6:
        return 'Recent (0-6 days)'
    elif days <= 13:
        return 'Mid-term (7-13 days)'
    else:
        return 'Early (14+ days)'

df['Timing_Cohort'] = df['Days_Before'].apply(timing_group)

In [6]:
conn = sqlite3.connect('event_data.db')
df.to_sql('registrations', conn, if_exists='replace', index=False)

print("Number of people per group:")
print(df['Timing_Cohort'].value_counts())

Number of people per group:
Timing_Cohort
Early (14+ days)        100
Recent (0-6 days)        50
Mid-term (7-13 days)     38
Name: count, dtype: int64


In [7]:
def run_sql(query, show_result=True):
    result = pd.read_sql_query(query, conn)

    if show_result:
        print("=" * 70)
        print("Query:")
        print("=" * 70)
        print(result)
        print()

    return result

print("  SQL Cohort: run_sql('SELECT * FROM registrations')")

  SQL Cohort: run_sql('SELECT * FROM registrations')


In [8]:
# Overall_Stats

query1 = """
SELECT
    COUNT(*) AS total_registered,
    SUM(Attended) AS total_attended,
    ROUND(AVG(Attended) * 100, 1) AS attendance_rate,
    ROUND((1 - AVG(Attended)) * 100, 1) AS no_show_rate
FROM registrations
"""

result1 = run_sql(query1, "QUERY 1: Overall_Stats")

Query:
   total_registered  total_attended  attendance_rate  no_show_rate
0               188              55             29.3          70.7



In [9]:
#By_Registration_Time

query2 = """
SELECT
    Timing_Cohort,
    COUNT(*) AS total,
    SUM(Attended) AS attended,
    ROUND(AVG(Attended) * 100, 1) AS attendance_rate,
    ROUND((1 - AVG(Attended)) * 100, 1) AS no_show_rate
FROM registrations
GROUP BY Timing_Cohort
ORDER BY attendance_rate DESC
"""

result2 = run_sql(query2, "QUERY 2: By_Registration_Time")

#Finding Best Timing Cohort
print(f" Best Timing: {result2.iloc[0]['Timing_Cohort']}")
print(f" Attendance_rate: {result2.iloc[0]['attendance_rate']:.1f}%\n")

Query:
          Timing_Cohort  total  attended  attendance_rate  no_show_rate
0     Recent (0-6 days)     50        23             46.0          54.0
1      Early (14+ days)    100        25             25.0          75.0
2  Mid-term (7-13 days)     38         7             18.4          81.6

 Best Timing: Recent (0-6 days)
 Attendance_rate: 46.0%



In [10]:
# By_Country (TOP 5)

query3 = """
SELECT
    Country,
    COUNT(*) AS total,
    SUM(Attended) AS attended,
    ROUND(AVG(Attended) * 100, 1) AS attendance_rate
FROM registrations
GROUP BY Country
HAVING COUNT(*) >= 10
ORDER BY total DESC
LIMIT 5
"""

result3 = run_sql(query3, "QUERY 3: By_Country (TOP 5)")

#Finding Lowest Attendance Rate
worst = result3.sort_values('attendance_rate').iloc[0]
print(f"Lowest Attendance Rate: {worst['Country']} ({worst['attendance_rate']:.1f}%)\n")

Query:
     Country  total  attended  attendance_rate
0      China     36        12             33.3
1      India     30         2              6.7
2  Australia     16         6             37.5
3      Nepal     14         3             21.4
4  Indonesia     13         6             46.2

Lowest Attendance Rate: India (6.7%)



In [11]:
# By_University (TOP 4)

query4 = """
SELECT
    University,
    COUNT(*) AS total,
    SUM(Attended) AS attended,
    ROUND(AVG(Attended) * 100, 1) AS attendance_rate
FROM registrations
GROUP BY University
ORDER BY total DESC
LIMIT 4
"""

result4 = run_sql(query4, "QUERY 4: By_University (TOP 4)")

Query:
             University  total  attended  attendance_rate
0                   UTS     43        12             27.9
1                  UNSW     37        10             27.0
2                  USYD     32        12             37.5
3  Macquarie University     14         3             21.4



In [12]:
# Cross_Analysis(Time x Country)

query5 = """
SELECT
    Timing_Cohort,
    Country,
    COUNT(*) AS total,
    SUM(Attended) AS attended,
    ROUND(AVG(Attended) * 100, 1) AS rate
FROM registrations
WHERE Country IN ('China', 'India', 'Australia')
GROUP BY Timing_Cohort, Country
HAVING COUNT(*) >= 3
ORDER BY rate DESC
LIMIT 10
"""

result5 = run_sql(query5, "QUERY 5: Cross_Analysis(Time x Country)")

Query:
          Timing_Cohort    Country  total  attended  rate
0  Mid-term (7-13 days)  Australia      5         3  60.0
1      Early (14+ days)      China     20         8  40.0
2     Recent (0-6 days)      China      5         2  40.0
3      Early (14+ days)  Australia      7         2  28.6
4     Recent (0-6 days)  Australia      4         1  25.0
5     Recent (0-6 days)      India      4         1  25.0
6  Mid-term (7-13 days)      China     11         2  18.2
7      Early (14+ days)      India     23         1   4.3
8  Mid-term (7-13 days)      India      3         0   0.0



In [13]:
#Area of Study (TOP 10)

query6 = """
SELECT
    "Area of Study" AS major,
    COUNT(*) AS total,
    SUM(Attended) AS attended,
    ROUND(AVG(Attended) * 100, 1) AS attendance_rate
FROM registrations
WHERE "Area of Study" IS NOT NULL
GROUP BY "Area of Study"
HAVING COUNT(*) >= 3
ORDER BY attendance_rate DESC  -- 참석률 높은 순
LIMIT 10
"""

result6 = run_sql(query6, "QUERY 6: Area of Study (TOP 10)")

Query:
                     major  total  attended  attendance_rate
0                Education      4         3             75.0
1       Project Management      7         4             57.1
2  Business Administration      8         4             50.0
3                Marketing      8         3             37.5
4                 Politics      3         1             33.3
5       Business Analytics     10         3             30.0
6                    Other      4         1             25.0
7                 Commerce      4         1             25.0
8                       IT     25         6             24.0
9              Engineering     13         3             23.1



In [14]:
# Statistical analysis
query7 = """
SELECT
    COUNT(*) AS total_count,
    COUNT(DISTINCT Country) AS unique_countries,
    COUNT(DISTINCT University) AS unique_universities,
    SUM(Attended) AS total_attended,
    AVG(Days_Before) AS avg_days_before,
    MIN(Days_Before) AS min_days,
    MAX(Days_Before) AS max_days
FROM registrations
"""

result7 = run_sql(query7, "QUERY 7: Statistical analysis")

Query:
   total_count  unique_countries  unique_universities  total_attended  \
0          188                30                   23              55   

   avg_days_before  min_days  max_days  
0        14.180851         0        34  



In [15]:
with pd.ExcelWriter('SQL_Anlysis.xlsx') as writer:
    result1.to_excel(writer, sheet_name='Overall_Stats', index=False)
    result2.to_excel(writer, sheet_name='By_Registration_Time', index=False)
    result3.to_excel(writer, sheet_name='By_Country', index=False)
    result4.to_excel(writer, sheet_name='By_University', index=False)
    result5.to_excel(writer, sheet_name='Cross_Analysis', index=False)
    result6.to_excel(writer, sheet_name='Area of Study', index=False)
    result7.to_excel(writer, sheet_name='Statistical analysis', index=False)

files.download('SQL_Anlysis.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>